In [3]:
import pandas as pd
import numpy as np

advetsing_data = pd.read_csv("Datasets/advertising_dataset.csv")
advetsing_data.head()

,user_id,timestamp,device_type,location,age_group,gender,ad_id,content_type,ad_topic,ad_target_audience,click_through_rate,conversion_rate,engagement_level,view_time,cost_per_click,click_through_rate.1,conversion_rate.1,ROI
0,896,2025-11-18 22:20:05,Tablet,UK,55+,Male,A5630,Image,Health,Young Adults,0.010588,0.0657,Ignored,21,0.35,0.010588,0.0657,0.20
1,827,2025-11-12 05:00:19,Desktop,Germany,45-54,Female,A6480,Text,Electronics,Family Oriented,0.022195,0.0237,Ignored,28,0.15,0.022195,0.0237,0.35
2,211,2025-10-22 05:21:08,Tablet,India,18-24,Female,A7800,Text,Health,Travel Lovers,0.081724,0.1055,Commented,47,1.28,0.081724,0.1055,0.67
3,897,2025-03-02 14:17:27,Desktop,USA,25-34,Male,A1514,Video,Health,Fitness Lovers,0.123995,0.0332,Liked,55,1.20,0.123995,0.0332,0.34
4,293,2025-07-25 09:47:22,Desktop,Germany,25-34,Male,A8366,Image,Health,Tech Enthusiasts,0.044757,0.1258,Shared,40,0.67,0.044757,0.1258,0.84
